# compact version of google customer spends
## ops : data cleanup and visualization

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import winsound
import json
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
#winsound.PlaySound('bell.wav', winsound.SND_FILENAME)
def load_df(csv_path='./train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

    df = pd.read_csv(csv_path, dtype={'fullVisitorId': 'str'}, nrows=nrows)

    for column in JSON_COLUMNS:
        df = df.join(pd.DataFrame(df.pop(column).apply(pd.io.json.loads).values.tolist(), index=df.index))

    return df
df = load_df()
df.drop(columns='adwordsClickInfo',axis=1,inplace=True)
keep_columns = df.columns[df.nunique()>1]
df = df.loc[:,keep_columns].copy()
df.columns


Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'browser', 'deviceCategory',
       'isMobile', 'operatingSystem', 'city', 'continent', 'country', 'metro',
       'networkDomain', 'region', 'subContinent', 'hits', 'pageviews',
       'transactionRevenue', 'adContent', 'campaign', 'keyword', 'medium',
       'referralPath', 'source'],
      dtype='object')

In [2]:
def add_time_features(df):
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='ignore')
    df['year'] = df['date'].apply(lambda x: x.year)
    df['month'] = df['date'].apply(lambda x: x.month)
    df['day'] = df['date'].apply(lambda x: x.day)
    df['weekday'] = df['date'].apply(lambda x: x.weekday())
    return df
df = add_time_features(df)

In [4]:
df['date'].describe()
df['year'].describe()
df['month'].describe()
df['day'].describe()
df['weekday'].describe()

count                  903653
unique                    366
top       2016-11-28 00:00:00
freq                     4807
first     2016-08-01 00:00:00
last      2017-08-01 00:00:00
Name: date, dtype: object

count    903653.000000
mean       2016.517079
std           0.499708
min        2016.000000
25%        2016.000000
50%        2017.000000
75%        2017.000000
max        2017.000000
Name: year, dtype: float64

count    903653.000000
mean          6.987671
std           3.484576
min           1.000000
25%           4.000000
50%           7.000000
75%          10.000000
max          12.000000
Name: month, dtype: float64

count    903653.000000
mean         15.697570
std           8.823674
min           1.000000
25%           8.000000
50%          16.000000
75%          23.000000
max          31.000000
Name: day, dtype: float64

count    903653.000000
mean          2.735183
std           1.929072
min           0.000000
25%           1.000000
50%           3.000000
75%           4.000000
max           6.000000
Name: weekday, dtype: float64

As to explain above steps please check google customer spends file.
In this notebook we will do extensive visualization and data cleaning operations.

In [3]:
# df['transactionRevenue'].fillna(0,inplace=True)
# df['transactionRevenue'] = df['transactionRevenue'].convert_objects(convert_numeric=True)

In [4]:
# for i in range(df.shape[1]) :
#     if(df[df.columns[i]].dtypes == 'O') :
#         df[df.columns[i]] = df[df.columns[i]].convert_objects(convert_numeric=True)

In [5]:
# for i in range(df.shape[1]) :
#     if(df[df.columns[i]].count() != 903653) :
#         print(df[df.columns[i]].name)
#         print(df[df.columns[i]].describe())

we can convert some of this to numeric.

In [2]:
df['transactionRevenue'] = df['transactionRevenue'].convert_objects(convert_numeric=True)
df['pageviews'] = df['pageviews'].convert_objects(convert_numeric=True)

In [3]:
df['transactionRevenue'].describe()

count    1.151500e+04
mean     1.337448e+08
std      4.482852e+08
min      1.000000e+04
25%      2.493000e+07
50%      4.945000e+07
75%      1.076550e+08
max      2.312950e+10
Name: transactionRevenue, dtype: float64

In [7]:
# df['transactionRevenue'].describe()
# df['pageviews'].describe()

In [8]:
df['transactionRevenue'].fillna(0,inplace=True)

In [9]:
df.to_csv('./df.csv',index=False)

In [10]:
# pageview vs other parameters.
df1 = df

In [11]:
df1.shape
df1.columns

(903653, 27)

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'browser', 'deviceCategory',
       'isMobile', 'operatingSystem', 'city', 'continent', 'country', 'metro',
       'networkDomain', 'region', 'subContinent', 'hits', 'pageviews',
       'transactionRevenue', 'adContent', 'campaign', 'keyword', 'medium',
       'referralPath', 'source'],
      dtype='object')

In [12]:
df['pageviews'].describe()
df1['transactionRevenue'].describe()

count    903553.000000
mean          3.849764
std           7.025274
min           1.000000
25%           1.000000
50%           1.000000
75%           4.000000
max         469.000000
Name: pageviews, dtype: float64

count    9.036530e+05
mean     1.704273e+06
std      5.277866e+07
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.312950e+10
Name: transactionRevenue, dtype: float64

In [13]:
df1.dropna(axis=0,subset=['pageviews'], inplace=True)

KeyboardInterrupt: 

In [ ]:
for i in range(df1.shape[1])
plt.scatter(df1['pageviews'], df1['transactionRevenue'])
plt.xlabel("pageviews")
plt.ylabel("transactionRevenue")
#plt.axis([, , -1, 0.03])
plt.grid(True)
plt.show()

In [ ]:
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
sns.distplot(df1['pageviews'],kde=False);

In [ ]:
sns.distplot(df1['transactionRevenue'],kde=True);

In [ ]:
pd.options.display.max_rows = 30
df1['pageviews'].value_counts()